In [1]:
import xml.etree.ElementTree as et
import textwrap as tw
import pandas as pd
from termcolor import colored
import re
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
import json
import matplotlib.pyplot as plt
import os
from string import punctuation

In [2]:
path = 'doc/'
files = os.listdir(path)
ls_xml = []

for i in files:
    if '.xml' in i:
        ls_xml.append(i)

print(ls_xml)

['test1.xml', 'test2.xml', 'test3.xml']


In [3]:
def get_xml_content(ls_xml, path, df):
    
    for xml in ls_xml:
        tree = et.parse(path + xml)
        root = tree.getroot()
        ct_at =0        
        
        for x in root:
            ct_as =0
            # print('xml:',xml,'ct_at:', ct_at)
            
            if x.tag == 'PubmedArticle':
                
                # Article的位置不固定
                cnt = 0
                for child in root[ct_at][0]:
                    if child.tag == 'Article':
                        break
                    cnt += 1
                
                for y in root[ct_at][0][cnt]:
                    # print(y.tag)
                    title = ''
                    abstract = ''
                    
                    if y.tag == 'ArticleTitle':
                        title = y.text
                        # print('Title:',title)
                        df.loc[len(df)] = [xml, ct_at, title, abstract]

                    elif y.tag == 'Abstract':
                        for z in root[ct_at][0][cnt][ct_as].findall('AbstractText'):
                            temp = tw.fill(z.text, initial_indent = '  ')
                            temp = temp + '\n'
                            abstract += temp

                        # print(abstract)
                        df.loc[len(df)-1, 'Content'] = abstract          
                        break
                        
                    ct_as += 1
                    
            ct_at += 1
    return df

In [4]:
columns = ["XML","ID","Title","Content"]
df = pd.DataFrame(columns = columns)
df01 = pd.DataFrame(columns = columns)
df02 = pd.DataFrame(columns = columns)

df = get_xml_content(ls_xml, path, df)
print('df.head(10):',df.head(10))

df.head(10):          XML ID                                              Title  \
0  test1.xml  0               Treatment for COVID-19: An overview.   
1  test2.xml  0  Epidemiology of COVID-19 infection in young ch...   
2  test3.xml  0  Neuropsychiatric aspects of COVID-19 pandemic:...   

                                             Content  
0    Coronavirus disease 2019 (COVID-19) is an in...  
1    Emerging evidence suggests young children ar...  
2    Corona virus disease (COVID-19) has been dec...  


In [5]:
def compare(f01, f02):
    
    df01 = df[df['XML'] == f01]
    cont01 = df01.iloc[0]['Content']
    # print(cont01)
    df02 = df[df['XML'] == f02]
    cont02 = df02.iloc[0]['Content']
    # print(cont01)
    
    if cont01 == cont02:
        print('The 2 files are the same.')
    else:
        sent01 = sent_tokenize(cont01)
        sent02 = sent_tokenize(cont02)
        s_len = min([len(sent01), len(sent02)])

        for i in range(0, s_len):
            if sent01[i] != sent02[i]:
                s01, s02 = sent01[i], sent02[i]
                word01 = word_tokenize(s01)
                word02 = word_tokenize(s02)
                w_len = min([len(word01), len(word02)])
                
                for j in range(0, w_len):
                    if word01[j] != word02[j]:
                        ind = j                            
                        break
                        
                colored_text = []
                for k in range(0, len(word01)):
            
                        if k == ind:
                            colored_text.append(colored(word01[k], 'grey','on_yellow'))

                        else:
                            colored_text.append(word01[k])

                s01 = " ".join(colored_text)
                colored_text = []

                for k in range(0, len(word02)):
            
                        if k == ind:
                            colored_text.append(colored(word02[k], 'grey','on_yellow'))

                        else:
                            colored_text.append(word02[k])

                s02 = " ".join(colored_text)
                colored_text = []
                break
                
        print('\nThe 2 files are different, starting from sentences: \n[file01]', s01, '\n[file02]', s02)

In [6]:
files = input('2 files to compare (seperated by a space): ')
files = re.split(r'(\s)',files)
while ' ' in files:
    files.remove(' ')
f01, f02 = files[0], files[1]

compare(f01, f02)

2 files to compare (seperated by a space): test1.xml test2.xml

The 2 files are different, starting from sentences: 
[file01] Coronavirus disease 2019 ( COVID-19 ) is an infectious disease caused by coronavirus-2 ( SARS-CoV-2 ) that causes a severe acute respiratory syndrome , a characteristic hyperinflammatory response , vascular damage , microangiopathy , angiogenesis and widespread thrombosis . 
[file02] Emerging evidence suggests young children are at greater risk of COVID-19 infection than initially predicted .
